In [14]:
import json 
import pathlib

In [24]:
mesh_dir = "sftp://paard/home/tlips/Documents/synthetic-cloth-data/synthetic-cloth-data/data/deformed_meshes/TSHIRT/dsd-tshirts"
mesh_dir = pathlib.Path(mesh_dir)
print(mesh_dir)

sftp:/paard/home/tlips/Documents/synthetic-cloth-data/synthetic-cloth-data/data/deformed_meshes/TSHIRT/dsd-tshirts


In [26]:
meshes = mesh_dir.glob("*.obj")
jsons = mesh_dir.glob("*.json")

In [28]:
print(len(list(jsons)))

0


In [30]:
for file in jsons:
    json_dict = json.load(file.open())
    coverage = json_dict["area"] / json_dict["flat_mesh"]["area"]
    print(coverage)